In [1]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

Loading configurations from /workspaces/fundamentals-of-data-transformation-4554074/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaylimit,10
displaycon,False


,Count
0,224


Ok, now we'll have some fun and dig into _aggregations_ in SQL! Let's start with our first... The most basic, `COUNT`!

In [2]:
%%sql
SELECT
    COUNT(*) as num_parks
FROM nps_public_data.parks p
WHERE designation = 'National Park'

,num_parks
0,51


In [3]:
%%sql
SELECT
    *
FROM nps_public_data.parks 
LIMIT 5;

,relevanceScore,designation,weatherInfo,addresses,operatingHours,entrancePasses,name,description,directionsUrl,fees,...,activities,url,longitude,id,images,directionsInfo,fullName,parkCode,latLong,latitude
0,1,National Memorial,http://forecast.weather.gov/MapClick.php?CityN...,"[{'type': 'Physical', 'line2': '', 'line1': '1...","[{'name': 'Hours of Operation', 'standardHours...",[],Federal Hall,"Here on Wall Street, George Washington took th...",http://www.nps.gov/feha/planyourvisit/directio...,[],...,"[{'name': 'Arts and Culture', 'id': '09DF0950-...",https://www.nps.gov/feha/index.htm,-74.010256,2337D255-2D32-4997-957A-D461EEA03AF8,[{'url': 'https://www.nps.gov/common/uploads/s...,The main entrance of Federal Hall is located a...,Federal Hall National Memorial,feha,"lat:40.70731192, long:-74.01025636",40.707312
1,1,National Historic Trail,"In winter, watch for ice on trails and sidewal...","[{'type': 'Physical', 'line2': '', 'line1': '6...","[{'name': 'Visitor Center Hours', 'standardHou...",[],Lewis & Clark,The Lewis and Clark National Historic Trail wi...,https://www.nps.gov/lecl/,[],...,"[{'name': 'Auto and ATV', 'id': '5F723BAD-7359...",https://www.nps.gov/lecl/index.htm,-95.924515,5D443C5F-19A0-4A06-9CE4-30534A3DD81A,[{'url': 'https://www.nps.gov/common/uploads/s...,Lewis & Clark National Historic Trail Headquar...,Lewis & Clark National Historic Trail,lecl,"lat:41.2646141052, long:-95.9245147705",41.264614
2,1,,"Summers are generally hot and humid, with dayt...","[{'type': 'Physical', 'line2': '', 'line1': '1...",[{'name': 'National Capital Parks-East Headqua...,[],National Capital Parks-East,Welcome to National Capital Parks-East. We inv...,http://www.nps.gov/nace/planyourvisit/directio...,[],...,"[{'name': 'Biking', 'id': '7CE6E935-F839-4FEC-...",https://www.nps.gov/nace/index.htm,-76.994000,BA3C1A1D-AA6A-49EB-9237-0222CEEE2670,[{'url': 'https://www.nps.gov/common/uploads/s...,DC295 South to the Exit for I-694/I-395/Capito...,National Capital Parks-East,nace,"lat:38.8659, long:-76.994",38.865900
3,1,National Historical Park,"Be prepared for hot, humid weather. The histor...","[{'type': 'Physical', 'line2': '', 'line1': '1...","[{'name': 'Visitor Center', 'standardHours': {...",[{'description': 'Adams National Historical Pa...,Adams,From the sweet little farm at the foot of Penn...,http://www.nps.gov/adam/planyourvisit/directio...,[],...,"[{'name': 'Guided Tours', 'id': 'B33DC9B6-0B7D...",https://www.nps.gov/adam/index.htm,-71.011604,E4C7784E-66A0-4D44-87D0-3E072F5FEF43,[{'url': 'https://www.nps.gov/common/uploads/s...,"Traveling on U.S. Interstate 93, take exit 7 -...",Adams National Historical Park,adam,"lat:42.2553961, long:-71.01160356",42.255396
4,1,Memorial Parkway,Summers on the parkway are generally hot and h...,"[{'type': 'Physical', 'line2': '700 George Was...",[{'name': 'George Washington Memorial Parkway ...,[],George Washington,The George Washington Memorial Parkway was des...,http://www.nps.gov/gwmp/planyourvisit/directio...,[],...,"[{'name': 'Arts and Culture', 'id': '09DF0950-...",https://www.nps.gov/gwmp/index.htm,-77.149500,E6D5BB41-3251-469F-ABDA-7B43B966F0CF,[{'url': 'https://www.nps.gov/common/uploads/s...,Directions to Parkway Headquarters From the so...,George Washington Memorial Parkway,gwmp,"lat:38.9628, long:-77.1495",38.962800


In [5]:
%%sql
SELECT
    states
    
FROM nps_public_data.parks p
WHERE designation = 'National Park'

LIMIT 20;

,states
0,MI
1,CO
2,AZ
3,OR
4,"ID,MT,WY"
5,WA
6,TX
7,NM
8,MO
9,ME


We can also count by state:

In [4]:
%%sql
SELECT
    states,
    COUNT(*) as num_parks
FROM nps_public_data.parks p
WHERE designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 20;

,states,num_parks
0,CA,5
1,UT,5
2,WA,3
3,FL,3
4,AZ,3
5,CO,3
6,NM,2
7,HI,2
8,SD,2
9,TX,2


But we note that states are defined as comma separate strings, so we'll need to split those strings to get a list of states for each park. We can do that with:

In [6]:
%%sql
SELECT
    fullname,
    UNNEST(
        SPLIT(states, ',')::string[]
        ) as state
FROM nps_public_data.parks p
WHERE designation = 'National Park'
LIMIT 5

,fullName,state
0,Isle Royale National Park,MI
1,Black Canyon Of The Gunnison National Park,CO
2,Grand Canyon National Park,AZ
3,Crater Lake National Park,OR
4,Yellowstone National Park,ID


This _will_ produce multiple records for the same park (those in multiple states), but that's what we want! Now we can count the number of parks across state lines using an aggregation

In [7]:
%%sql
WITH park_list AS (
    SELECT
        fullname,
        UNNEST(
            SPLIT(states, ',')::string[]
            ) as state
    FROM nps_public_data.parks p
    WHERE designation = 'National Park'
)
SELECT
    state,
    COUNT(*) as num_parks
FROM park_list
GROUP BY 1
ORDER BY 2 DESC, 1
LIMIT 10

,state,num_parks
0,CA,6
1,UT,5
2,AZ,3
3,CO,3
4,FL,3
5,WA,3
6,AK,2
7,HI,2
8,MT,2
9,NM,2


Again, note a _pattern_: in order to count what we needed, we had to derive _one row per state per park_. Our original query gave us one row per park with mulitple states. So we _transformed our data_ as an intermediate step towards our final end.

This intermediate transformation is common in obtaining the format of data we need to perform our aggregation:

1. Inspect data and determine your aggregation, i.e. counting parks by state
2. Note the current _shape_ or format, i.e. at the state level
3. Note the _desired_ format, i.e. at the state-park level
4. Transform to get to the desired format, then aggregate

How do we find the campgrounds with the least and most sites using aggregations?

- Obtain the total number of campsites for each campground (that's a national park)
- Get the min / max numbers
- Filter the results by those numbers

In [8]:
%%sql
WITH park_campgrounds AS (
    SELECT
        c.name as campgroud_name,
        p.fullname as park_name,
        c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as total_sites,
    FROM nps_public_data.campgrounds c
    INNER JOIN nps_public_data.parks p
        ON c.parkcode = p.parkcode
        AND p.designation = 'National Park'
    GROUP BY 1,2,3
), min_max_sites AS (
SELECT
    MIN(total_sites) as min_sites,
    MAX(total_sites) as max_sites
FROM park_campgrounds
WHERE total_sites > 0
)
SELECT
    campgroud_name,
    total_sites as num_sites,
    CASE total_sites WHEN min_sites THEN 'least' ELSE 'most' END as sites_rank
FROM park_campgrounds pc
INNER JOIN min_max_sites mms
    ON (pc.total_sites = mms.min_sites OR pc.total_sites = mms.max_sites)
ORDER BY num_sites, campgroud_name

,campgroud_name,num_sites,sites_rank
0,Dunbar Group Site,1,least
1,Frijole Horse Corral Campground,1,least
2,Group Campsite,1,least
3,Hay Bay Campground,1,least
4,Pickerel Cove Campground,1,least
5,Roundup Group Horse Camp,1,least
6,Upper Goodell Creek Group Campsite,1,least
7,Morefield Campground,534,most


What about the parks? We can follow similar logic

In [9]:
%%sql
WITH park_campgrounds AS (
    SELECT
        c.name as campgroud_name,
        p.fullname as park_name,
        c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as total_sites,
    FROM nps_public_data.campgrounds c
    INNER JOIN nps_public_data.parks p
        ON c.parkcode = p.parkcode
        AND p.designation = 'National Park'
    GROUP BY 1,2,3
), park_sites AS (
    SELECT
        park_name,
        SUM(total_sites) as num_sites
    FROM park_campgrounds
    GROUP BY 1
    ORDER BY 2 DESC
), min_max_sites AS (
    SELECT
        MIN(num_sites) as min_sites,
        MAX(num_sites) as max_sites
    FROM park_sites ps
)
SELECT
    ps.*,
    CASE num_sites WHEN min_sites THEN 'least' ELSE 'most' END as sites_rank
FROM park_sites ps
INNER JOIN min_max_sites mms
    ON (num_sites = mms.min_sites or num_sites = mms.max_sites)
ORDER BY ps.num_sites DESC

,park_name,num_sites,sites_rank
0,Yellowstone National Park,2149.0,most
1,Dry Tortugas National Park,8.0,least


Another important aggregation pattern is _using boolean logic within aggregations_ as filters! For example

- `COUNT DISTINCT`
- `COUNT CASE WHEN`

This lets us filter our count _without_ filtering the underlying data.

In [10]:
%%sql
SELECT
    p.fullname as park_name,
    -- COUNT the number of campgrounds
    COUNT(DISTINCT c.name) as num_campgrounds,
    -- Get the average number of sites— what is this returning?
    AVG(numberofsitesreservable) as avg_sites_reservable,
    AVG(numberofsitesfirstcomefirstserve) as avg_sites_fcfs,
    AVG(numberofsitesreservable + numberofsitesfirstcomefirstserve) as avg_total_sites
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
-- Read more about aggregates here: https://duckdb.org/docs/sql/aggregates

,park_name,num_campgrounds,avg_sites_reservable,avg_sites_fcfs,avg_total_sites
0,Isle Royale National Park,36,1.194444,5.583333,6.777778
1,Yosemite National Park,14,75.642857,21.642857,97.285714
2,Glacier National Park,13,64.384615,13.307692,77.692308
3,Guadalupe Mountains National Park,13,8.461538,0.000000,8.461538
4,Olympic National Park,13,36.846154,23.307692,60.153846
5,Great Smoky Mountains National Park,13,72.538462,0.000000,72.538462
6,Death Valley National Park,12,11.333333,52.083333,63.416667
7,Yellowstone National Park,12,179.083333,0.000000,179.083333
8,North Cascades National Park,11,26.636364,7.818182,34.454545
9,Joshua Tree National Park,9,47.000000,8.555556,55.555556


In [11]:
%%sql
SELECT
    p.name as park_name,
    COUNT(CASE WHEN c.numberofsitesreservable > 0 THEN 1 END) as num_reservable_campgrounds,
    COUNT(CASE WHEN c.numberofsitesfirstcomefirstserve > 0 THEN 1 END) as num_fcfs_campgrounds,
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5

,park_name,num_reservable_campgrounds,num_fcfs_campgrounds
0,Isle Royale,17,36
1,Guadalupe Mountains,13,0
2,Great Smoky Mountains,13,0
3,Yosemite,12,4
4,Yellowstone,12,0


There's lots more to aggregations, but they're pretty simple. The best way to get started will be to aggregate data in practice. Remember the basics:
- Aggregations collapse rows.
- Rows that _aren't_ being aggregated must be `GROUP`ed.
- `GROUP` statements appear at the end of a query.
- Duplicate rows may skew aggregates if not properly accounted for.